In [5]:
!pip install setuptools

In [1]:
!pip install -r requirements.txt

  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached keras-3.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.29.0-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached setuptools-75.6.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached termcolor-2.5.0-py3-

In [1]:
# Tengo un dataset en data/train/ que son imágenes para entrenar un modelo de clasificación de redes neuronales convolucionales
# Quiero que el modelo sea capaz de clasificar imágenes de gatos y perros

# 1. Cargar las imágenes
# 2. Preprocesar las imágenes
# 3. Crear el modelo
# 4. Entrenar el modelo
# 5. Evaluar el modelo
# 6. Guardar el modelo
# 7. Hacer predicciones

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers, models


In [2]:
train_dir = "data/train"
test_dir = "data/test"

# Carga los datasets
train_dataset = image_dataset_from_directory(
    train_dir,
    labels='inferred',         # Las etiquetas se infieren del nombre de la carpeta
    label_mode='int',          # Etiquetas como enteros
    image_size=(128, 128),     # Redimensiona las imágenes a 128x128
    batch_size=32,             # Tamaño del lote
    shuffle=True               # Mezcla los datos
)

test_dataset = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='int',
    image_size=(128, 128),
    batch_size=32,
    shuffle=False
)
# Inspección de los datos
class_names = train_dataset.class_names
print(f"Clases detectadas: {class_names}")

Found 4733 files belonging to 2 classes.
Found 1184 files belonging to 2 classes.
Clases detectadas: ['chihuahua', 'muffin']


In [3]:

# Opcional: Normalización de imágenes (valores entre 0 y 1)
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))



# Opcional: Configuración para prefetching (mejora el rendimiento durante el entrenamiento)
# AUTOTUNE = tf.data.AUTOTUNE
# train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
# test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)


In [5]:
# Definir el modelo de la CNN
model = models.Sequential([
    # Primera capa convolucional
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),

    # Segunda capa convolucional
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Tercera capa convolucional
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Aplanar y agregar capas densas
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),

    # Capa de salida (número de clases)
    layers.Dense(len(class_names), activation='softmax')  # len(class_names) = número de clases
])

# Compilar el modelo
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
model.summary()

# Entrenar el modelo
model.fit(train_dataset, epochs=10)

# Evaluar el modelo
loss, accuracy = model.evaluate(test_dataset)
print(f"Pérdida: {loss}")
print(f"Exactitud: {accuracy}")


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,313,026 (12.64 MB)

 Trainable params: 3,313,026 (12.64 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 153ms/step - accuracy: 0.6235 - loss: 0.6836
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 22s 146ms/step - accuracy: 0.8250 - loss: 0.4081
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 22s 146ms/step - accuracy: 0.8684 - loss: 0.3284
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 22s 148ms/step - accuracy: 0.8892 - loss: 0.2550
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 22s 149ms/step - accuracy: 0.9259 - loss: 0.1950
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 23s 152ms/step - accuracy: 0.9368 - loss: 0.1544
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 22s 150ms/step - accuracy: 0.9582 - loss: 0.1017
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 23s 154ms/step - accuracy: 0.9726 - loss: 0.0868
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 159ms/step - accuracy: 0.9837 - loss: 0.0563
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 22s 147ms/step - accuracy: 0.9843 - loss: 0.0522
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - accuracy: 0.9276 - loss: 0.3149
Pérdida: 0.47092756628990173
Exacti